# Choosing spectra to make them pass through the LSST's effective passbands 

In this notebook we are going to modify the format of the [spectral atlases contained within pysynphot](https://www.stsci.edu/hst/instrumentation/reference-data-for-calibration-and-tools/astronomical-catalogs.html) to better work with them in the following notebooks. 

This notebook requires [pysynphot](https://pysynphot.readthedocs.io/en/latest/). 

# Load necessary packages 

In [14]:
import numpy as np
import matplotlib.pylab as plt
import pysynphot as S
from astropy.io import fits
import pandas as pd
import os

# Paths to the possible spectral atlases 
For the moment, we are only considering observeded spectra, not models 

In [2]:
pysynphot_root_path=os.environ['PYSYN_CDBS']
path_dict = {
    'calspec':os.path.join(pysynphot_root_path,'calspec'),
    'bpgs':os.path.join(pysynphot_root_path,'grid/bpgs'),
    'jacoby':os.path.join(pysynphot_root_path,'grid/jacobi'),
    'pickles':os.path.join(pysynphot_root_path,'grid/pickles/dat_uvk')
}

In [3]:
info_dict = {
    'calspec':'',
    'bpgs':'BPGS_README',
    'jacoby':'jspectype.fits',
    'pickles':'pickles_uk.fits'
}

In [4]:
outdir = 'spectral_atlases/'
if os.path.exists(outdir)==False:
    os.mkdir(outdir)

# Select spectral atlas 
For the moment, we are only considering observeded spectra, not models 

In [34]:
# Options: 
# calspec (HST calibration spectra): work in progress, need to separate stars from AGNs and other sources 
# bpgs (Bruzual-Persson-Gunn-Stryker): 175 spectra 
# jacobi (Jacoby-Hunter-Christian): 161 spectra 
# pickles (Pickles): 131 spectra 
atlas_name = 'jacoby'
atlas_path = path_dict[atlas_name]
atlas_info = info_dict[atlas_name]

In [35]:
data_list = []
if atlas_name=='calspec':
    pass
elif atlas_name=='bpgs':
    readme = open(os.path.join(atlas_path,atlas_info),'r').readlines()
    print('Number of spectra = ', len(readme[3:]))
    for i in range(3,len(readme)):
        info = readme[i].split()
        file = info[0][:-4]+'.fits'
        source_name = info[1]
        if len(info)==3:
            spec_type = info[2]
        else:
            spec_type = 'unknown'
        teff = 'unknown'
        
        data_list.append([file,spec_type,teff,source_name])
        
elif atlas_name=='jacoby':
    readme = fits.open(os.path.join(atlas_path,atlas_info))
    info = readme[1].data
    print('Number of spectra = ', len(info))
    for i in range(len(info)):
        file = info[i][0].lower()+'.fits'
        spec_type = info[i][1]
        source_name = 'unknown'
        teff = 'unknown'
        #print(file,spec_type)
        
        data_list.append([file,spec_type,teff,source_name])
    
elif atlas_name=='pickles':
    readme = fits.open(os.path.join(atlas_path,atlas_info))
    info = readme[1].data
    print('Number of spectra = ', len(info))
    #print(info[0])
    
    temp_file = open(os.path.join(outdir,'pickles.txt'),'r').readlines()
    file_ = []
    spec_type_ = []
    teff_ = []
    for line in temp_file:
        file_.append(line.split()[0])
        spec_type_.append(line.split()[1])
        teff_.append(line.split()[2])
    
    for i in range(len(info)):
        file = info[i][0]+'.fits'
        spec_type = info[i][1]
        source_name = 'unknown'
        #print(file,spec_type)
        if info[i][0] in file_:
            index = np.where(np.array(file_)==info[i][0])[0][0]
            #print(spec_type,spec_type_[index])
            assert (spec_type==spec_type_[index].replace('-',''))
            teff = teff_[index]
        else:
            teff = None
        
        data_list.append([file,spec_type,teff,source_name])
    
data_list = np.array(data_list)
data_frame = pd.DataFrame(data_list, columns=['File','Type','Teff','Name'])
data_frame.to_csv(outdir+'atlas_{0}.txt'.format(atlas_name),index=False)

Number of spectra =  161
